In [1]:
# reading DARDAR .h5 files and outputting monthly files for totals 
#  and counts onto a chosen grid, which can then be read by another
#  python script to create a multi-month mean grid
# dduncan, 14/12/17
# updated 2/1/18 for doing day and night separately (or both in future, i guess)
# 8/1/18 made vertical version for getting profile information (on EC p grid)

import h5py
from mpl_toolkits.basemap import Basemap, cm
from pyhdf.SD import SD, SDC
from isa import isa # intl std atmosphere routine in src/pyco/isa.py

In [2]:

dir = '/home/dudavid/Dendrite/Dendrite/SatData/DARDAR/'
yrs = ['15'] #08','09'] # day and night data years on dendrite
dn = 'day'
startlat = 90.0 # abs value of NS boundaries
gsize = 2.5 # grid size in degree
nx, ny = int(360/gsize), int(startlat*2/gsize)
nz = 436 # total vertical levels in DARDAR #about how many are levels are >0m and <16km (~100hPa)
nzd= 418-151 #indices for 0m and 16020 in Dardar data, respectively
# i.e. 1000-975, 975-950... 150-100
#plvs = [1013.25,1000,975,950,925,900,875,850,825,800,775,750,700,650,600,550,\
#       500,450,400,350,300,250,225,200,175,150,100] #should be 26 plevs
plvs = [1030,1000,975,950,925,900,875,850,825,800,775,750,700,650,600,550,\
        500,450,400,350,300,250,200,150,100] #shrink by a couple to p levels common to ERA/MERRA
plvs = plvs[::-1]
nnz= len(plvs)-1 # 25 between-layer levels? coming from EC 1000-100hPa levels from ERA5

# try doing 6 pixel long average of DARDAR data
nsmo = 0 #3 # double this so 6? -- [ x-nsmo : n+nsmo ]

In [3]:
for yr in yrs:
    #yr = '09'
    print('starting year 20'+yr)
    dirt = dir+'20'+yr+'/'
    months = sorted(os.listdir(dirt))
    print(months)
    for m in months[0:]: #[0:1]:
        #if os.path.isfile(outfi):
        #    print(outfi,' already exists!')
        #else:
        print('month: ',m)
        iwcg,iwcgc = np.zeros([nnz,ny,nx]), np.zeros([ny,nx]) # re-init for each month
        iwcgd = np.zeros([nzd,ny,nx])#, np.zeros([ny,nx]) # re-init for each month
        #iwpall = []
        days = sorted(os.listdir(dirt+m+'/'))
        ndays = 0 #init
        for d in days:
            print(d)
            files = sorted(os.listdir(dirt+m+'/'+d))
            if len(files) != 0:  # in case no files in day dir
                ndays += 1
                for fix in files: #range(len(files)):
                    file = dirt+m+'/'+d+'/'+fix
                    #print(file)

                    if yr == '15' or yr == '08':
                        f = h5py.File(file,'r')
                        iw = np.array(f['iwc']) # IWC in kg/m^3
                        lat = np.array(f['latitude']) #[mask]
                        lon = np.array(f['longitude']) # [mask]
                        dnf = np.array(f['day_night_flag']) # night=1, day=0
                    else:
                        f = SD(file,SDC.READ)
                        iw = f.select('iwc')[:,:]
                        lat = f.select('latitude')[:]
                        lon = f.select('longitude')[:]
                        dnf = f.select('day_night_flag')[:] # night=1, day=0
                    #print(np.shape(iw),np.shape(lat))
                    #flag = np.array(f['land_water_mask'])
    # mask is from calipso flags where:
    # (http://www.icare.univ-lille1.fr/projects_data/dardar/docs/varcloud-algorithm_description-v1.0.pdf)
    #  0=shallow ocean 1=land 2=coastlines 3=shallow inland water 4=intermittent water 
    #  5=deep inland water 6=coastal ocean 7=deep ocean
                    if( m == months[0] and d == days[0] and fix == files[0] ): # don't calc every time (always the same)
                        if yr=='15' or yr=='08': hgt = np.array(f['height']) #height above sea level in m [436]
                        if yr!='15' and yr!='08': hgt = f.select('height')[:] #height above sea level in m [436]
                        hmask = []
                        #ecsubs = np.zeros([nnz+1])
                        dsubs = np.zeros([nz])-1
                        ct = 0
                        for z in range(len(hgt)):
                            if hgt[z]>0 and hgt[z]<16000:  
                                hmask.append(z) #saves index values for bins 0-16km alt
                                # input alt, get pressure
                                ptemp = isa(hgt[z])
                                #print(z,hgt[z],ct,ptemp)
                                if ptemp < 100*plvs[ct+1] and ptemp >= 100*plvs[ct]: dsubs[z] = ct
                                if isa(hgt[z+1]) > 100*plvs[ct+1] and isa(hgt[z+1]) < 100*plvs[ct+2]: ct+=1
                            
                    #mask = flag >= 0 # all!
                    #mask = flag > 4 # ocean and deep inland water

                    if dn == 'day': mask = np.array([dnf == 0])[0,:] # DAYTIME ONLY
                    if dn == 'night': mask = np.array([dnf == 1])[0,:] # NIGHTTIME ONLY

                    iwc = iw[mask,:] # just selected profiles
                    lat = lat[mask]
                    lon = lon[mask]
                    #iwc = iw[:,:] # all
                    if len(lat) > 10: # a few orbits have zero length, so skip them
                    
                    # new part for _vert (8/1/17), average into ERA5 vertical levels
                        iwc_temp = np.zeros([len(lat),nnz])
                        iwc_tempd =np.zeros([len(lat),nzd])
                        for ez in range(nnz):
                             if len(dsubs==ez) > 0:
                                 iwc_temp[:,ez] = np.mean(iwc[:,dsubs==ez],axis=1) # take avg of all D obs within EC levels
                         #temp = iwc[:,dsubs==ez]
                        #iwc_temp[:,ez] = np.mean(iwc[:,dsub],axis=1) # take avg of all D obs within EC levels
                        iwc_tempd = iwc[:,151:418]

                    #dmsk = np.array(f['DARMASK_Simplified_Categorization'])[mask] 
                    #as follows (same source doc):
                    # -9 ground, -1 dunno, 0 clear, 1 ice, 2 ice+supercooled, 3 liquid (warm),
                    #  4 supercooled, 5 rain, 6 aerosol, 7 maybe insects, 8 stratospheric feature
                    #dayn = np.array(f['day_night_flag'])[mask] #day=0, night=1

                    # calculate lat/lon indices for Xdeg grid (fixed 23/11):
                        ladex = np.round((lat[:]+startlat-gsize*.5) / gsize)
                        lodex = np.round((lon[:]+180.0-gsize*.5) / gsize)

                    #iwp = [sum(iwc[x,hmask]*60.0) for x in range(len(iwc[:,20]))]
                        ariwc = np.array(iwc_temp[:,:])
                    # all range gates are 60m here, so kg/m3 * m yields IWP in kg/m2

                    # running mean over iwp series:
                        iwcsmoo = np.array(ariwc) # first few will be same, so will last few. 
                        iwcsmood= np.array(iwc_tempd) # first few will be same, so will last few. 
                    #for ez in range(nnz):
                        if nsmo > 0:
                            iwcsmoo[nsmo:(len(lat)-nsmo),:] = \
                         [np.mean(ariwc[(x-nsmo):(x+nsmo),:],axis=0) for x in range(nsmo,(len(lat)-nsmo))]
                    #for ez in range(nzd):
                            iwcsmood[nsmo:(len(lat)-nsmo),:] = \
                         [np.mean(iwc_tempd[(x-nsmo):(x+nsmo),:],axis=0) for x in range(nsmo,(len(lat)-nsmo))]
                        #else:
                        #    iwcsmoo = ariwc
                        #    iwcsmood= iwc_tempd

                        for x in range(nx):
                          if np.any(lodex == x):
                            dexsub = np.where(lodex == x)
                            subset = iwcsmoo[dexsub,:][0,:,:] # using smoothed iwp
                            subsetd= iwcsmood[dexsub,:][0,:,:] # using smoothed iwp
                            subsetla = ladex[dexsub]
                            for y in range(ny):
                                if np.any(subsetla == y):
                                    lolasubset = np.where(subsetla == y) # [1,X]
                                    temp = subset[lolasubset,:][0,:,:] # [X,nz]
                                    iwcg[:,y,x] += temp.sum(axis=0) # add up, get mean afterward
                                    iwcgc[y,x]  += np.shape(temp)[0] #len(subset[lolasubset,0])
                                    #print(np.shape(subset[lolasubset,:])[1],np.shape(temp)[0])
                                    #print(np.shape(subset),np.shape(lolasubset),np.shape(temp))
                                    tempd = subsetd[lolasubset,:][0,:,:]
                                    iwcgd[:,y,x] += tempd.sum(axis=0) # add up, get mean afterward
                                    #print(subset[:,:,:,32,:])


        # create histogram of IWP
        #binz = [2**x for x in range(-14,5)]
        #hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
        nd = str(ndays).rjust(2,'0')
        # dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
        fin = 'grids/dardar.nsmo0.'+dn+'.'+str(nx)+'x.'+str(ny)+'y.iwc.v2.'+yr+m+'.'+nd
        #fin = 'grids/dardar.'+str(nx)+'x.'+str(ny)+'y.iwptot.cts.hist.v1.'+yr+m+'.'+nd
        np.save(fin+'.tot',iwcg) #should be [25,72,144] array of reals
        np.save(fin+'.totd',iwcgd) #should be bigger array of reals
        np.save(fin+'.cts',iwcgc) #just [72,144] since counts for all vertical levels ==


starting year 2015
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month:  01
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  02
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  03
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  04
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  05
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  06
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  07
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  08
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
month:  09
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22